In [7]:
using PlotRecipes
gr(ma=0.5)

Plots.GRBackend()

In [8]:
ex = :(
function add_ast(adjlist, names, ex::Expr, parent_idx)
   idx = length(names)+1
   push!(names, string(ex.head))
   push!(adjlist, Int[add_ast(adjlist, names, arg, idx) for arg in ex.args])
   idx
end
);

In [18]:
plot(ex, size=(800,800), root=:left, ms=1.5)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#da6f93; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 186.892,378.161 193.372,377.851 198.758,376.951 203.13,375.504 206.571,373.554 209.161,371.145 210.98,368.321 212.111,365.124 212.635,361.6 212.631,357.792 
 212.182,353.743 211.368,349.497 210.27,345.098 208.97,340.59 207.548,336.016 206.086,331.42 204.664,326.847 203.364,322.338 202.266,317.939 201.452,313.694 
 201.003,309.645 200.999,305.836 201.522,302.312 202.653,299.116 204.473,296.291 207.063,293.882 210.503,291.933 214.876,290.486 220.262,289.586 226.741,289.276 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#da6f93; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 186.892,378.161 192.789,378.404 197.815,379.111 202.034,380.247 205.512,381.778 208.312,383.67 210.5,385.888 212.139,388.398 213.295,391.166 214.032,394.157 
 214.413,397.336 214.505,400.671 214.37,404.125 214.075,407.665 213.682,411.257 213.258,414.866 212.865,418.458 212.57,421.999 212.436,425.453 212.527,428.787 
 212.909,431.967 213.645,434.958 214.801,437.726 216.44,440.236 218.628,442.454 221.428,444.345 224.906,445.877 229.126,447.013 234.151,447.72 240.048,447.963 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#da6f93; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 226.741,289.276 231.327,289.366 235.388,289.627 238.963,290.047 242.091,290.613 244.811,291.313 247.161,292.133 249.181,293.061 250.91,294.085 252.386,295.191 
 253.648,296.366 254.735,297.599 255.687,298.877 256.541,300.186 257.336,301.514 258.113,302.848 258.909,304.176 259.763,305.486 260.714,306.763 261.801,307.996 
 263.063,309.172 264.539,310.277 266.268,311.301 268.288,312.229 270.639,313.049 273.358,313.749 276.486,314.315 280.061,314.735 284.122,314.996 288.708,315.086 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#da6f93; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 226.741,289.276 232.522,289.288 237.686,289.324 242.279,289.381 246.347,289.459 249.936,289.555 253.091,289.667 255.858,289.794 258.283,289.934 260.41,290.086 
 262.287,290.246 263.958,290.415 265.47,290.59 266.867,290.769 268.196,290.951 269.502,291.134 270.831,291.316 272.228,291.495 273.74,291.67 275.411,291.839 
 277.288,292 279.415,292.151 281.84,292.291 284.607,292.418 287.762,292.53 291.351,292.626 295.419,292.704 300.012,292.761 305.176,292.797 310.957,292.809 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#da6f93; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 226.741,289.276 231.857,289.189 236.397,288.937 240.405,288.532 243.924,287.986 246.995,287.312 249.662,286.521 251.967,285.626 253.953,284.64 255.661,283.574 
 257.136,282.44 258.419,281.252 259.553,280.02 260.58,278.758 261.544,277.478 262.486,276.192 263.45,274.911 264.477,273.649 265.611,272.418 266.894,271.229 
 268.368,270.096 270.077,269.03 272.063,268.043 274.368,267.149 277.034,266.358 280.106,265.684 283.624,265.138 287.633,264.733 292.173,264.481 297.288,264.394 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#da6f93; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 226.741,289.276 234.441,288.984 241.063,288.137 246.69,286.775 251.399,284.94 255.271,282.672 258.386,280.013 260.823,277.005 262.662,273.687 263.983,270.103 
 264.865,266.291 265.389,262.295 265.634,258.154 265.679,253.911 265.604,249.605 265.49,245.279 265.416,240.974 265.461,236.731 265.705,232.59 266.229,228.593 
 267.111,224.782 268.432,221.197 270.271,217.88 272.708,214.871 275.823,212.213 279.695,209.945 284.405,208.11 290.031,206.748 296.654,205.901 304.353,205.609 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#da6f93; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 226.741,289.276 232.25,289.098 237.046,288.582 241.181,287.752 244.709,286.634 247.683,285.253 250.154,283.633 252.177,281.801 253.804,279.78 255.088,277.596 
 256.081,275.274 256.836,272.839 257.407,270.317

In [19]:
png("/tmp/tmp")